### Строим логистическую регрессию - угадываем пол спортсмена по признакам

https://www.kaggle.com/rio2016/olympic-games

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv( 'athletes.csv' )
data.head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
0,736041664,A Jesus Garcia,ESP,male,10/17/69,1.72,64.0,athletics,0,0,0
1,532037425,A Lam Shin,KOR,female,9/23/86,1.68,56.0,fencing,0,0,0
2,435962603,Aaron Brown,CAN,male,5/27/92,1.98,79.0,athletics,0,0,1
3,521041435,Aaron Cook,MDA,male,1/2/91,1.83,80.0,taekwondo,0,0,0
4,33922579,Aaron Gate,NZL,male,11/26/90,1.81,71.0,cycling,0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11538 entries, 0 to 11537
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           11538 non-null  int64  
 1   name         11538 non-null  object 
 2   nationality  11538 non-null  object 
 3   sex          11538 non-null  object 
 4   dob          11537 non-null  object 
 5   height       11208 non-null  float64
 6   weight       10879 non-null  float64
 7   sport        11538 non-null  object 
 8   gold         11538 non-null  int64  
 9   silver       11538 non-null  int64  
 10  bronze       11538 non-null  int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 991.7+ KB


#### Попробуем угадать пол на основе роста, веса и вида спорта

Посмотрим много ли в наших признаках пустых значений

In [4]:
data[ pd.isnull( data['height'] ) ].head()

,id,name,nationality,sex,dob,height,weight,sport,gold,silver,bronze
12,258556239,Abbas Qali,IOA,male,10/11/92,NaN,NaN,aquatics,0,0,0
47,469953606,Abdoullah Bamoussa,ITA,male,6/8/86,NaN,NaN,athletics,0,0,0
50,325809293,Abdul Omar,GHA,male,10/3/93,NaN,NaN,boxing,0,0,0
52,262868423,Abdulaziz Alshatti,IOA,male,10/30/90,NaN,NaN,fencing,0,0,0
56,897549624,Abdullah Hel Baki,BAN,male,8/1/89,NaN,NaN,shooting,0,0,0


In [5]:
print 'Для height пустых строк {}'.format( len( data[ pd.isnull( data['height'] ) ] ) )
print 'Для weight пустых строк {}'.format( len( data[ pd.isnull( data['weight'] ) ] ) )
print 'Для sport пустых строк {}'.format( len( data[ pd.isnull( data['sport'] ) ] ) )
print 'Всего строк в наборе {}'.format( len( data ) )

Для height пустых строк 330
Для weight пустых строк 659
Для sport пустых строк 0
Всего строк в наборе 11538


In [5]:
data['height'].unique()

array([1.72, 1.68, 1.98, 1.83, 1.81, 1.8 , 2.05, 1.93, 1.65, 1.7 , 1.75,
        nan, 1.61, 1.78, 1.76, 2.1 , 1.73, 1.85, 1.77, 1.9 , 1.86, 1.74,
       1.6 , 2.07, 1.88, 1.66, 1.62, 1.87, 2.03, 1.69, 1.82, 1.89, 1.94,
       1.95, 1.71, 1.84, 1.91, 1.67, 2.02, 1.58, 1.63, 1.79, 1.97, 1.56,
       1.55, 1.57, 1.46, 1.92, 1.64, 1.53, 1.99, 1.96, 2.  , 2.04, 1.47,
       1.52, 2.01, 1.51, 1.59, 2.08, 1.37, 1.5 , 1.45, 2.06, 1.54, 2.11,
       1.43, 1.49, 1.33, 1.48, 1.44, 2.13, 2.09, 2.21, 2.18, 1.21, 1.38,
       1.34, 2.15, 2.17, 1.42, 1.4 , 2.14])

In [6]:
# обнуляем ячейки с отсутствием данных
data = data[ pd.isnull( data['height'] ) == 0 ]
data = data[ pd.isnull( data['weight'] ) == 0 ]

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10858 entries, 0 to 11537
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           10858 non-null  int64  
 1   name         10858 non-null  object 
 2   nationality  10858 non-null  object 
 3   sex          10858 non-null  object 
 4   dob          10858 non-null  object 
 5   height       10858 non-null  float64
 6   weight       10858 non-null  float64
 7   sport        10858 non-null  object 
 8   gold         10858 non-null  int64  
 9   silver       10858 non-null  int64  
 10  bronze       10858 non-null  int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 1017.9+ KB


In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
# попробуем выбрать какие-нибудь признаки
selectedColumns = data[ [ 'height', 'sport', 'sex' ] ]

# столбец sport является категориальной переменой
# переведем ее в значения 0 и 1, добавив столбцы с соответствующими названиями
X = pd.get_dummies( selectedColumns, columns = [ 'sport' ] )

# столбец sex является целевой переменной, удаляем его из X
del X['sex']
X.head()

,height,sport_aquatics,sport_archery,sport_athletics,sport_badminton,sport_basketball,sport_canoe,sport_cycling,sport_equestrian,sport_fencing,...,sport_rugby sevens,sport_sailing,sport_shooting,sport_table tennis,sport_taekwondo,sport_tennis,sport_triathlon,sport_volleyball,sport_weightlifting,sport_wrestling
0,1.72,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.68,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1.98,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.83,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1.81,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# целевая переменная (столбец sex) снова является категориальной
# переведем значения столбца в числа, оставив один столбец

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [11]:
le.fit( data['sex'] )

LabelEncoder()

In [12]:
le.classes_

array(['female', 'male'], dtype=object)

In [15]:
# пример "расшировки" столбца sex

le.transform( [ 'male', 'female', 'male' ] )

array([1, 0, 1], dtype=int64)

In [13]:
# записываем в переменную y преобразованный столбец sex

y = pd.Series( data = le.transform( data['sex'] ) )
y.head()

0    1
1    0
2    1
3    1
4    1
dtype: int32

In [14]:
model = LogisticRegression()

In [15]:
# обучаем модель

model.fit( X, y )
predictions = model.predict_proba( X )

In [16]:
predictions[:5]

array([[0.60928184, 0.39071816],
       [0.84036981, 0.15963019],
       [0.01413978, 0.98586022],
       [0.32454756, 0.67545244],
       [0.14087571, 0.85912429]])

In [20]:
# сравниваем факт с предсказаниями
# (да, сравнивать предсказания с самими данными не круто, лучше было разбить на обучающую и тестовую выборку)
# это будет в следующем занятии

list(zip( predictions[:, 1], y )) #дает положительные результаты predict_proba

[(0.39071816288456934, 1),
 (0.15963018588847486, 0),
 (0.9858602181273184, 1),
 (0.6754524361458484, 1),
 (0.8591242853049126, 1),
 (0.7697704478555832, 1),
 (0.9427362446620023, 1),
 (0.9305283872329617, 1),
 (0.7307475043433045, 0),
 (0.15359636342748179, 0),
 (0.30896282180795814, 1),
 (0.5241618823532392, 1),
 (0.08106048915588161, 0),
 (0.4724648231563249, 0),
 (0.5516510406195377, 0),
 (0.297483415368585, 1),
 (0.5959186412917049, 1),
 (0.5959186412917049, 0),
 (0.9759391500677925, 1),
 (0.43439164520190815, 1),
 (0.8699044211643581, 1),
 (0.6123974594127148, 1),
 (0.7727769331922296, 1),
 (0.9427735267899271, 1),
 (0.5241618823532392, 1),
 (0.6060563452919243, 1),
 (0.8995178671028996, 1),
 (0.914684113505773, 1),
 (0.8995178671028996, 1),
 (0.6304846335751443, 1),
 (0.8889904723686914, 1),
 (0.8636304037270733, 1),
 (0.30896282180795814, 1),
 (0.6123974594127148, 1),
 (0.37444188381082094, 1),
 (0.9150344992923596, 1),
 (0.5461144714519158, 1),
 (0.29444638224645797, 1),
 (0.5

In [18]:
model.score(X, y)

0.7966476330815988